In [1]:
#This script does two things: 
#(1) creates a dataframe from BED results for visualization 
#(2) Adds features: family motif, signature mutation class

#import Bio
import math
#import pyranges as pr
from pyjaspar import jaspardb
import numpy as np
import pandas as pd
import dask.dataframe as dd
import multiprocessing

In [2]:
import sys
!{sys.executable} -m pip install utils

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/comp/python/3.9.5/rackham/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
import utils

In [3]:
import pandas as pd
def check(mutate, row):
    info=mutate.split("*")
    startC=int(info[0])
    endC=int(info[1])
    if row[2]>=startC and row[1]<=endC:
        return True
    else:
        return False

def filtering(ori_bed, output):
    for i in range(0, ori_bed.shape[0]):
        seq_info=ori_bed.iloc[i, 0].split("|")
        for mutate in seq_info[1:]:
            if check(mutate, ori_bed.iloc[i,:]):
                row=[str(ele) for ele in ori_bed.iloc[i, :]]
                row='\t'.join(row)
                output.write(f'{row}\n')
                break
    return 

In [7]:
    file1="chr21.filtered.bed"
    file2="chr21.Filtered.bed"
    ori_bed=pd.read_csv(file1, header=None, delimiter='\t')
    file2 = file2.split('/')[-1]
    output=open(file2, 'w')
    output.close()
    output=open(file2, 'a')
    filtering(ori_bed, output)
    output.close()

In [1]:
#Entropy Auxilary function
def entropy_l(lst):
    sm = 0
    for i in range(0,len(lst)):
        if lst[i]!=0:
            sm+=-lst[i]*math.log2(lst[i])
    return 2-sm

In [5]:
from pyjaspar import jaspardb

In [6]:
jdb_obj = jaspardb(release='JASPAR2020')

In [7]:
motifs = jdb_obj.fetch_motifs(
    collection = 'CORE',
    tax_group = ['vertebrates']
)


In [8]:
#compute mutation entropy value.
#Give the motif and position of mutation returns change in entropy
def entropy_mut(pos,mot):
    return(entropy_l([dc_pwm[mot][0][pos],dc_pwm[mot][1][pos],dc_pwm[mot][2][pos],dc_pwm[mot][3][pos]]))

In [14]:
"""""""""""""""""""""""""""""""""""""""""

    #Compute mutational signature of the SNP
    df_["Mut_Sig"]=''

    #C>A
    # evaluate the condition 
    condition = df_.eval("Wld=='C' and Mut=='A' or Wld=='G' and Mut=='T'")
    # assign values
    df_['Mut_Sig'] = np.where(condition, "C>A",df_.Mut_Sig)


    #C<G
    condition = df_.eval("Wld=='C' and Mut=='G' or Wld=='G' and Mut=='C'")
    # assign values
    df_['Mut_Sig'] = np.where(condition, "C>G",df_.Mut_Sig)

    #C<T
    condition = df_.eval("Wld=='C' and Mut=='T' or Wld=='G' and Mut=='A'")
    # assign values
    df_['Mut_Sig'] = np.where(condition, "C>T",df_.Mut_Sig)

    #T<A
    condition = df_.eval("Wld=='T' and Mut=='A' or Wld=='A' and Mut=='T'")
    # assign values
    df_['Mut_Sig'] = np.where(condition, "T>A",df_.Mut_Sig)

    #T<C
    condition = df_.eval("Wld=='T' and Mut=='C' or Wld=='A' and Mut=='G'")
    # assign values
    df_['Mut_Sig'] = np.where(condition, "T>C",df_.Mut_Sig)

    #T<G
    condition = df_.eval("Wld=='T' and Mut=='G' or Wld=='A' and Mut=='C'")
    # assign values
    df_['Mut_Sig'] = np.where(condition, "T>G",df_.Mut_Sig)
"""""""""""""""""""""""""""""""""""""""""

'""\nfor i in range(0,len(df_i)):\n        info = df_i[i].split(\'|\')\n        if len(info)>2:\n            st = int(df_s[i])\n            end = int(df_e[i])\n            hits = 0 #how many of the mutations overlap with motif discovered, if more than 1 overlap we leave it there.\n            for j in range(1,len(info)):\n                mps=int(info[j].split(\'*\')[0])\n                if st<mps and mps<end:\n                    hits=+1\n                    if hits>1:\n                        df_i[i] = info[0]+ str(\'|\')+str(\'e\')+info[j]\n                    df_i[i] = info[0]+ str(\'|\') + info[j]\n                    \n    for i in range(0,len(m_id)):\n        try:\n            m = jdb_obj.fetch_motif_by_id(m_id[i])\n            if df_.mSta[i][0]==\'e\':\n                ent = 2\n            else:\n                ps = int(df_.mSta[i])-int(df_.Start[i])\n                ent = entropy_mut(ps,m)\n            \n            ent_l.append(ent)\n            cl_l.append(m.tf_class[0])\n  

In [9]:
#returns correct info label, CHECK CHECK CHECK
def info_pr(inf,st,end):
    info = inf.split('|')
    if len(info)>2:
            st = int(st)
            end = int(end)
            hits = 0 #how many of the mutations overlap with motif discovered, if more than 1 overlap we leave it there.
            for j in range(1,len(info)):
                mps=int(info[j].split('*')[0])
                if st<mps and mps<end:
                    hits=+1
                    if hits>1:
                        #tells us if the motif has overlapping mutations
                        return(nfo[0]+ str('|')+str('e')+info[j])
                    return(info[0]+ str('|') + info[j])

SyntaxError: invalid syntax (<ipython-input-49-34c59602f05c>, line 1)

In [10]:
#for each specific motif returns entropy
def info_mot(mot,msta,start):
    ent = 0
    if mot in dc_pwm:
        #if more than 2 mutations produced motif flag
        if msta[0]=='e':
            ent = 2
            return (ent)
        elif int(msta)>=int(start):
            try:
                ps = int(msta)-int(start)
                ent = entropy_mut(ps,mot)
                return (ent)
            except:
                return(0)
        else:
            return(0)
    else:
        return('MISS')


In [11]:
def motif_class(mot):
    return (dc_cl[mot])

In [12]:
def motif_fam(mot):
    return (dc_fam[mot])

In [23]:
d = 'hi'

In [297]:
type(d)=='str'

False

In [25]:
global 

SyntaxError: invalid syntax (<ipython-input-25-5efaa5e110c5>, line 1)

In [22]:
def bed2df(file):

    df = file

    #This script checks if when there are multiple sites
    df_i= df['Chromosome'].tolist()
    df_s = df['Start'].tolist()
    df_e = df['End'].tolist()
    #from multiprocessing import Pool
    pool = multiprocessing.Pool(4)
    infol = pool.starmap(info_pr, zip(df_i,df_s,df_e))
    df_i.append(infol)
    pool.close()
    #print(df_i[0:4])
    df['Chromosome']= pd.Series(df_i)
    
    df['Chromosome'] = pd.Series(df_i)
    df[['Chr', 'Info']] = df['Chromosome'].str.split('|',1, expand=True)
    df1 = df['Info'].str.split('*',6, expand=True)
    #drop rows with more than 1 mutation for region
    
    #print('done with desc. manipulation')
    #print(df1.head(3))
    #clean columns
    #df1.drop(df1.columns[[*range(7,12)]], axis=1, inplace=True)
    #df_a = df.loc[df1.index]
    print('done with desc. manipulation')
    #print('dfa')
    #df_ = pd.merge(df, df1, on="key")
    df_= pd.merge(df, df1, right_index=True, left_index=True)
    #df = df_.drop(['Info','Chromosome'], axis=1)
    #print('df_')
    #print(df_.head(3))
    
    
    #rename columns
    col_nms = ['mSta', 'mEnd','Wld', 'Mut','Canc_T', 'Mut_T', 'Pat_ID']
    df_.columns = df_.columns[:9].tolist() + col_nms
    df_[['Name','M_ID']]= df_['Name'].str.split('_', expand=True)
    
    #df_ = df_.reset_index()
    #print('df_')
    #print(df_.head(3))
    
    df_ = df_.drop(df_[df_.Mut_T=='INS'].index).copy(deep=True)
    #df_ = df_.reset_index()
    df_omut = df_[df_['Mut_T']!='SNP'].copy(deep=True)
    #df_omut = df_omut.reset_index()
    #Get TF class category and entropy diff
    cl_l =[]
    fm_l =[]
    ent_l = []
    
    
    
    
    #df_= df_[df_['Mut_T']!='SNP'].copy(deep=True)
    #df_omut = df_omut.reset_index(drop=True)
    
    #remove insertions because they are possibly wrong
    df_omut = df_omut.loc[df_omut['Mut_T']!='INS']
    df_omut = df_omut.reset_index(drop=True)
    #print('other')
    #print(df_omut.head(3))
    
    #filter by entropy motifs and concatanate with other mutations
    dfsnp = df_[df_['Mut_T']=='SNP']
    dfsnp = dfsnp.reset_index(drop=True)
    
    #print('SNP')
    #print(dfsnp.head(3))
    
    m_id = dfsnp['M_ID'].tolist()
    df_ms = dfsnp['mSta'].tolist()
    df_s = dfsnp['Start'].tolist()
    print('starting with entropy...')
    
    pool = multiprocessing.Pool(4)
    entro = pool.starmap(info_mot, zip(m_id,df_ms,df_s))
    pool.close()
    

    ent_l.append(entro)
    print(ent_l)
    dfsnp["Entropy"]= ent_l[0]
    dfsnp["Entropy"] = pd.to_numeric(dfsnp["Entropy"])
    #dfsnp.insert(7, "Entropy", pd.Series(ent_l))
    #print('SNP')
    #print(dfsnp.head(3))
    dfsnp = dfsnp[dfsnp["Entropy"]>1.6]
    dfsnp = dfsnp.drop(["Entropy"], axis=1)
    #dfsnp = dfsnp.reset_index(drop=True)
    #dfsnp = dfsnp.drop(["Entropy"], axis=1)
    #df_omut = df_omut.drop(["level_0"], axis=1)
    #df_omut = df_omut.reset_index(drop=True)
    
    #print('Other')
    #print(df_omut.head(3))
    
    #print('SNP')
    #print(dfsnp.head(3))
    
    df = pd.concat([df_omut, dfsnp],ignore_index = True).reset_index(drop=True)
    df = df.drop(['Info','mSta','mEnd' ], axis=1)
    
    #cl_l =[]
    #fm_l =[]
    
    pool = multiprocessing.Pool(4)
    m_id = df['M_ID'].tolist()
    cl = pool.map(motif_class, m_id)
    cl_l.append(cl)
    print(len(cl_l))
    df['Class']= pd.Series(cl_l[0])
    fm = pool.map(motif_fam, m_id)
    fm_l.append(fm)
    df['Fam']= pd.Series(fm_l[0])
    
    pool.close
    #datf[['Tissue', 'Cancer_T']] = datf['Canc_T'].str.split('-', 1, expand=True)
    #df.to_csv('data/p1.whole_gen.filt.uniq.df', sep ='\t', index=False)
    return (df)
    

In [ ]:
dc_pwm

In [23]:
if __name__=="__main__":
    global dc_pwm
    global dc_cl
    global dc_fam
    #inpf = sys.argv[1]
    inpf = "chr21.Filtered.bed"
    #bed=pd.read_csv(inpf, header=None, delimiter='\t')
    d_f = dd.read_csv(inpf,header=None, delimiter='\t')
    dafr =d_f.compute()
    #df = gr.df
    dafr = dafr.rename(columns={ 0 : "Chromosome", 1: "Start", 2: "End", 3: "Name", 4: "Score", 5: "pval", 6: "Strand"})
    #dafr = dafr.drop(columns=['Score', 'pval','Strand'])
    dafr = bed2df(dafr)
    #datf.to_csv('data/chr5.filt_mot_df.bed', sep ='\t', index=False)
    #outf = 'data/chr21.filt_uniq_motifs.bed'
    #inpf, outf= sys.argv[1], sys.argv[2]
    #output=open(outf, 'w')
    #output.close()
    #output=open(outf, 'a')
    #bed2df(file)
    #output.close()

done with desc. manipulation
starting with entropy...
[[1.5453499172857166, 1.4953872553622445, 1.9251265594108293, 1.8915945270140848, 0.47333414750740843, 1.952632945037037, 1.4006717767694377, 1.4076953291076364, 0.9444885341662053, 1.573270118358041, 0.5657582865328019, 0.0, 0.244325638857617, 1.210856010314337, 1.9877304553282331, 1.9390134990879575]]
1


In [24]:
dafr

,Chromosome,Start,End,Name,Score,pval,Strand,Chr,Wld,Mut,Canc_T,Mut_T,Pat_ID,M_ID,Class,Fam
0,chr21|37797397*37797397*T*C*Liver-HCC*SNP*DO50848,37797397,37797407,Foxq1,15.2653,0.000003,-,chr21,T,C,Liver-HCC,SNP,DO50848,MA0040.1,Fork head / winged helix factors,Forkhead box (FOX) factors
1,chr21|18927369*18927369*G*C*Liver-HCC*SNP*DO50848,18927355,18927370,KLF9,15.7886,0.000002,-,chr21,G,C,Liver-HCC,SNP,DO50848,MA1107.2,C2H2 zinc finger factors,Three-zinc finger Kruppel-related factors
2,chr21|40509873*40509873*C*T*Liver-HCC*SNP*DO50848,40509870,40509881,Mafb,13.8315,0.000008,-,chr21,C,T,Liver-HCC,SNP,DO50848,MA0117.2,Basic leucine zipper factors (bZIP),Maf-related factors
3,chr21|19837427*19837427*T*G*Liver-HCC*SNP*DO50848,19837427,19837437,TCF7,13.7642,0.000005,-,chr21,T,G,Liver-HCC,SNP,DO50848,MA0769.2,High-mobility group (HMG) domain factors,TCF-7-related factors
4,chr21|16546740*16546740*A*G*Liver-HCC*SNP*DO50848,16546730,16546742,TEAD2,13.8364,0.000010,-,chr21,A,G,Liver-HCC,SNP,DO50848,MA1121.1,TEA domain factors,TEF-1-related factors


In [42]:
dafr

,Chromosome,Start,End,Name,Chr,Wld,Mut,Canc_T,Mut_T,Pat_ID,M_ID,Class,Fam
0,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,132973,132980,BARX1,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA0875.1,Homeo domain factors,NK-related factors
1,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,132973,132980,BARX1,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA0875.1,Homeo domain factors,NK-related factors
2,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,62860354,62860361,DLX5,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA1476.1,Homeo domain factors,NK-related factors
3,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,62860354,62860361,DLX5,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA1476.1,Homeo domain factors,NK-related factors
4,chr10|158881*158892*TCCTTTATCCAT*-*Liver-HCC*D...,158883,158890,HOXB6,chr10,TCCTTTATCCAT,-,Liver-HCC,DEL,DO50844,MA1500.1,Homeo domain factors,HOX-related factors
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221240,chr13|62808506*62808506*A*T*Liver-HCC*SNP*DO23524,62808505,62808518,Arid5a,chr13,A,T,Liver-HCC,SNP,DO23524,MA0602.1,ARID domain factors,ARID-related factors
221241,chr13|62817352*62817352*T*A*CNS-GBM*SNP*DO10908,62817333,62817353,IRF1,chr13,T,A,CNS-GBM,SNP,DO10908,MA0050.2,Tryptophan cluster factors,Interferon-regulatory factors
221242,chr13|62817352*62817352*T*A*CNS-GBM*SNP*DO10908,62817333,62817353,IRF1,chr13,T,A,CNS-GBM,SNP,DO10908,MA0050.2,Tryptophan cluster factors,Interferon-regulatory factors
221243,chr13|62826245*62826245*A*T*Lung-SCC*SNP*DO27334,62826231,62826245,MAFG,chr13,A,T,Lung-SCC,SNP,DO27334,MA0659.2,Basic leucine zipper factors (bZIP),Maf-related factors


In [15]:
#Create dictionaries of PWM, motif family and class some class and families have more than 2 categories, all have at least one class, Motif has no family:MA0111.1
#Motif has no family: MA0619.1 and 34 have more than one class and 67 have more than one family.
#pwm dic
dc_pwm = {}
#class dic
dc_cl =  {}
#fam dic
dc_fam = {}
for i in range(0,len(motifs)):
    dc_fam[motifs[i].matrix_id] = motifs[i].tf_family
    if len(motifs[i].tf_family)>=1:
        dc_fam[motifs[i].matrix_id] = motifs[i].tf_family[0]
    dc_cl[motifs[i].matrix_id] = motifs[i].tf_class
    if len(motifs[i].tf_class)>=1:
        dc_cl[motifs[i].matrix_id] = motifs[i].tf_class[0]
    dc_pwm[motifs[i].matrix_id] = motifs[i].pwm
    
    

In [ ]:
for i in range(0,len)

In [211]:
dafr['Chromosome'][0]

'chr19|100969*100975*CTATTTC*-*Panc-AdenoCA*DEL*DO51493'

In [45]:
dafr

,Chromosome,Start,End,Name,Chr,Wld,Mut,Canc_T,Mut_T,Pat_ID,M_ID,Class,Fam
0,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,132973,132980,BARX1,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA0875.1,Homeo domain factors,NK-related factors
1,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,132973,132980,BARX1,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA0875.1,Homeo domain factors,NK-related factors
2,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,62860354,62860361,DLX5,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA1476.1,Homeo domain factors,NK-related factors
3,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,62860354,62860361,DLX5,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA1476.1,Homeo domain factors,NK-related factors
4,chr10|158881*158892*TCCTTTATCCAT*-*Liver-HCC*D...,158883,158890,HOXB6,chr10,TCCTTTATCCAT,-,Liver-HCC,DEL,DO50844,MA1500.1,Homeo domain factors,HOX-related factors
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221240,chr13|62808506*62808506*A*T*Liver-HCC*SNP*DO23524,62808505,62808518,Arid5a,chr13,A,T,Liver-HCC,SNP,DO23524,MA0602.1,ARID domain factors,ARID-related factors
221241,chr13|62817352*62817352*T*A*CNS-GBM*SNP*DO10908,62817333,62817353,IRF1,chr13,T,A,CNS-GBM,SNP,DO10908,MA0050.2,Tryptophan cluster factors,Interferon-regulatory factors
221242,chr13|62817352*62817352*T*A*CNS-GBM*SNP*DO10908,62817333,62817353,IRF1,chr13,T,A,CNS-GBM,SNP,DO10908,MA0050.2,Tryptophan cluster factors,Interferon-regulatory factors
221243,chr13|62826245*62826245*A*T*Lung-SCC*SNP*DO27334,62826231,62826245,MAFG,chr13,A,T,Lung-SCC,SNP,DO27334,MA0659.2,Basic leucine zipper factors (bZIP),Maf-related factors


In [214]:
dafr['Start'][0]

100974

In [ ]:
datf = bed2df(dafr)

In [24]:
dafr =d_f.compute()

In [25]:
dafr

,0,1,2,3,4,5,6
0,chr10|48588*48588*G*A*Eso-AdenoCa*SNP*DO50306,48584,48594,KLF10_MA1511.1,14.7551,0.000006,+
1,chr10|48588*48588*G*A*Eso-AdenoCa*SNP*DO50306,48584,48593,KLF5_MA0599.1,14.9388,0.000006,+
2,chr10|49643*49643*G*A*Breast-AdenoCa*SNP*DO3840,49636,49643,LMX1A_MA0702.2,12.9434,0.000027,-
3,chr10|49643*49643*G*A*Breast-AdenoCa*SNP*DO3840,49636,49643,LHX5_MA1519.1,12.0000,0.000027,+
4,chr10|49643*49643*G*A*Breast-AdenoCa*SNP*DO3840,49636,49643,NKX6-1_MA0674.1,11.4746,0.000027,+
...,...,...,...,...,...,...,...
653331,chrY|56877153*56877153*G*T*Lung-SCC*SNP*DO27025,56877148,56877158,FOXA3_MA1683.1,13.6069,0.000001,-
653332,chrY|56877153*56877153*G*T*Lung-SCC*SNP*DO27025,56877148,56877158,FOXK2_MA1103.2,13.5764,0.000002,-
653333,chrY|56877494*56877494*C*T*Skin-Melanoma*SNP*D...,56877490,56877502,ZNF24_MA1124.1,15.0545,0.000003,+
653334,chrY|56880553*56880553*C*A*Liver-HCC*SNP*DO50806,56880546,56880555,RBPJ_MA1116.1,13.4310,0.000005,+


In [27]:
dafr

,0,1,2,3,4,5,6
0,chr10|48588*48588*G*A*Eso-AdenoCa*SNP*DO50306,48584,48594,KLF10_MA1511.1,14.7551,0.000006,+
1,chr10|48588*48588*G*A*Eso-AdenoCa*SNP*DO50306,48584,48593,KLF5_MA0599.1,14.9388,0.000006,+
2,chr10|49643*49643*G*A*Breast-AdenoCa*SNP*DO3840,49636,49643,LMX1A_MA0702.2,12.9434,0.000027,-
3,chr10|49643*49643*G*A*Breast-AdenoCa*SNP*DO3840,49636,49643,LHX5_MA1519.1,12.0000,0.000027,+
4,chr10|49643*49643*G*A*Breast-AdenoCa*SNP*DO3840,49636,49643,NKX6-1_MA0674.1,11.4746,0.000027,+
...,...,...,...,...,...,...,...
653331,chrY|56877153*56877153*G*T*Lung-SCC*SNP*DO27025,56877148,56877158,FOXA3_MA1683.1,13.6069,0.000001,-
653332,chrY|56877153*56877153*G*T*Lung-SCC*SNP*DO27025,56877148,56877158,FOXK2_MA1103.2,13.5764,0.000002,-
653333,chrY|56877494*56877494*C*T*Skin-Melanoma*SNP*D...,56877490,56877502,ZNF24_MA1124.1,15.0545,0.000003,+
653334,chrY|56880553*56880553*C*A*Liver-HCC*SNP*DO50806,56880546,56880555,RBPJ_MA1116.1,13.4310,0.000005,+


In [50]:
dafr

,Chromosome,Start,End,Name,Chr,Wld,Mut,Canc_T,Mut_T,Pat_ID,M_ID,Class,Fam
0,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,132973,132980,BARX1,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA0875.1,Homeo domain factors,NK-related factors
1,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,132973,132980,BARX1,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA0875.1,Homeo domain factors,NK-related factors
2,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,62860354,62860361,DLX5,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA1476.1,Homeo domain factors,NK-related factors
3,chr10|132965*132973*ATGTAAGAT*-*ColoRect-Adeno...,62860354,62860361,DLX5,chr10,ATGTAAGAT,-,ColoRect-AdenoCA,DEL,DO10631,MA1476.1,Homeo domain factors,NK-related factors
4,chr10|158881*158892*TCCTTTATCCAT*-*Liver-HCC*D...,158883,158890,HOXB6,chr10,TCCTTTATCCAT,-,Liver-HCC,DEL,DO50844,MA1500.1,Homeo domain factors,HOX-related factors
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221240,chr13|62808506*62808506*A*T*Liver-HCC*SNP*DO23524,62808505,62808518,Arid5a,chr13,A,T,Liver-HCC,SNP,DO23524,MA0602.1,ARID domain factors,ARID-related factors
221241,chr13|62817352*62817352*T*A*CNS-GBM*SNP*DO10908,62817333,62817353,IRF1,chr13,T,A,CNS-GBM,SNP,DO10908,MA0050.2,Tryptophan cluster factors,Interferon-regulatory factors
221242,chr13|62817352*62817352*T*A*CNS-GBM*SNP*DO10908,62817333,62817353,IRF1,chr13,T,A,CNS-GBM,SNP,DO10908,MA0050.2,Tryptophan cluster factors,Interferon-regulatory factors
221243,chr13|62826245*62826245*A*T*Lung-SCC*SNP*DO27334,62826231,62826245,MAFG,chr13,A,T,Lung-SCC,SNP,DO27334,MA0659.2,Basic leucine zipper factors (bZIP),Maf-related factors


In [31]:
dafr = dafr.rename(columns={ 0 : "Chromosome", 1: "Start", 2: "End", 3: "Name", 4: "Score", 5: "pval", 6: "Strand"})  

In [23]:
names(d_f.compute())

NameError: name 'names' is not defined

In [40]:
datf[datf.Canc_T=='Panc-AdenoCA'].Fam.value_counts()

More than 3 adjacent zinc fingers            244
Three-zinc finger Kruppel-related            105
FOX                                          100
POU domain factors                            91
HOX                                           84
                                            ... 
Responders to external signals (SRF/RLM1)      2
GCNF-related receptors (NR6)                   2
THAP-related factors                           1
SMAD factors                                   1
HD-PROS factors                                1
Name: Fam, Length: 70, dtype: int64

In [43]:
datf.groupby(['Canc_T','Fam']).size().unstack(fill_value=0)

NameError: name 'datf' is not defined

In [48]:
analysis_d = datf.groupby(['Canc_T','Fam']).size().unstack(fill_value=0)

In [47]:
cl_l = dafr.Fam.tolist()
for i in range(0,len(cl_l)):
    if type(cl_l[i])==list:
        cl_l[i]='NA'


In [49]:
dafr["Fam"] = pd.Series(cl_l)

In [62]:
#df_fam_gen

In [63]:
#df_class_gen

In [57]:
df_fam_gen = dafr.groupby(['Canc_T','Fam']).size().unstack(fill_value=0)


In [58]:
df_fam_gen.to_csv('data/fam.motifs_type.csv')

In [53]:
df_class_gen = dafr.groupby(['Canc_T','Class']).size().unstack(fill_value=0)

In [54]:
df_class_gen.to_csv('data/class.motifs_type.csv')

In [50]:
#cancer type
analysis_d.to_csv('fam.motifs_type.csv')

In [53]:
#For Classification analysis
analysis_c.to_csv('class.motifs_type.csv')

In [52]:
analysis_c= datf.groupby(['Pat_ID','Class']).size().unstack(fill_value=0)

In [ ]:
datf.Pat_ID.

In [67]:
pat_fam = datf.groupby(['Canc_T','Pat_ID','Fam']).size().unstack(fill_value=0).sum(axis=1)

In [69]:
df_ml_fam = dafr.groupby(['Canc_T','Pat_ID','Fam']).size().unstack(fill_value=0)

In [66]:
#Dataframe with all transcription factors as features and patients with diagnosis as rows
df_ml_class=dafr.groupby(['Canc_T','Pat_ID','Class']).size().unstack(fill_value=0)

In [88]:
df_ml = datf.groupby(['Canc_T','Pat_ID','Fam']).size().unstack(fill_value=0)

In [68]:
df_ml_class.to_csv('ml.class.motifs_.csv')

In [70]:
df_ml_fam.to_csv('ml.fam.motifs_.csv')

In [83]:
pat_fam

Pat_ID
DO1000     3
DO1001    12
DO1002    25
DO1003    10
DO1004    22
          ..
DO9226    12
DO9700    12
DO9732    12
DO9788    13
DO9940     9
Length: 2182, dtype: int64

In [59]:
datf.Pat_ID

0        DO35152
1        DO35152
2        DO35152
3        DO46368
4        DO46368
          ...   
24471    DO46779
24472    DO50839
24473    DO45097
24474    DO37861
24475    DO34817
Name: Pat_ID, Length: 24476, dtype: object

In [611]:
df_omut.drop(['index','Chromosome','Info','Sta', ], axis=1)

,Start,End,Name,Score,Strand,ThickStart,Chr,Sta,End,Wld,Mut,Canc_T,Mut_T,Pat_ID,M_ID,Mut_Sig,Class,Ent,Fam
4,8524938,8524955,RARA,15.56060,1.32e-07,+,chr21,8524954,8524955,GA,AT,Stomach-AdenoCA,DNP,DO38671,MA0729.1,,Nuclear receptors with C4 zinc fingers,1.007318,Thyroid hormone receptor-related factors (NR1)
5,8524946,8524960,NR4A2::RXRA,15.13130,3.55e-06,-,chr21,8524954,8524955,GA,AT,Stomach-AdenoCA,DNP,DO38671,MA1147.1,,Nuclear receptors with C4 zinc fingers,0.427670,NGFI-B-related receptors (NR4)
6,8524946,8524960,NR1H4::RXRA,13.84400,8.07e-06,-,chr21,8524954,8524955,GA,AT,Stomach-AdenoCA,DNP,DO38671,MA1146.1,,Nuclear receptors with C4 zinc fingers,0.866383,Thyroid hormone receptor-related factors (NR1)
19,8692461,8692470,ZNF75D,15.17310,4.03e-06,-,chr21,8692460,8692462,CCA,ATT,Prost-AdenoCA,TNP,DO46813,MA1601.1,,C2H2 zinc finger factors,1.095706,More than 3 adjacent zinc fingers
41,8765944,8765959,PHOX2B,14.48030,5.58e-06,-,chr21,8765954,8765954,A,-,Head-SCC,DEL,DO15430,MA0681.2,,Homeo domain factors,0.239929,Paired-related HD factors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34570,46360362,46360373,ZNF384,14.74470,2e-06,-,chr21,46360339,46360367,CAGCCTCTCCGAGTAGCTGGGACTACAGG,-,Ovary-AdenoCA,DEL,DO30750,MA1125.1,,Rel homology region (RHR) factors,1.949405,None
34573,46371812,46371826,ZSCAN4,9.58621,9.74e-06,+,chr21,46371815,46371816,CA,-,Lung-SCC,DEL,DO26806,MA1155.1,,Basic helix-loop-helix factors (bHLH),0.201841,More than 3 adjacent zinc fingers
34583,46390949,46390960,YY1,15.11540,4.9e-06,+,chr21,46390956,46390956,G,-,Lung-AdenoCA,DEL,DO25985,MA0095.2,,C2H2 zinc finger factors,0.698564,More than 3 adjacent zinc fingers
34633,46448569,46448581,Stat2,14.24110,6.76e-06,-,chr21,46448578,46448578,T,-,Lung-SCC,DEL,DO26445,MA1623.1,,Fork head/winged helix factors,0.332357,STAT factors


In [615]:
#df_omut= datf[datf['Mut_T']!='SNP']
#df_omut = df_omut[df_omut['Mut_T']!='INS']

dfsnp =datf[datf['Mut_T']=='SNP']

dfsnp = dfsnp[dfsnp['Ent']>1.70]

df_ = pd.concat([dfsnp, df_omut])

In [616]:
df_

,index,Chromosome,Start,End,Name,Score,Strand,ThickStart,Chr,Info,...,mEn,Wld,Mut,Canc_T,Mut_T,Pat_ID,M_ID,Class,Ent,Fam
4,4,chr21|8524954*8524955*GA*AT*Stomach-AdenoCA*DN...,8524938,8524955,RARA,15.56060,1.32e-07,+,chr21,8524954*8524955*GA*AT*Stomach-AdenoCA*DNP*DO38671,...,8524955,GA,AT,Stomach-AdenoCA,DNP,DO38671,MA0729.1,None,None,None
5,5,chr21|8524954*8524955*GA*AT*Stomach-AdenoCA*DN...,8524946,8524960,NR4A2::RXRA,15.13130,3.55e-06,-,chr21,8524954*8524955*GA*AT*Stomach-AdenoCA*DNP*DO38671,...,8524955,GA,AT,Stomach-AdenoCA,DNP,DO38671,MA1147.1,None,None,None
6,6,chr21|8524954*8524955*GA*AT*Stomach-AdenoCA*DN...,8524946,8524960,NR1H4::RXRA,13.84400,8.07e-06,-,chr21,8524954*8524955*GA*AT*Stomach-AdenoCA*DNP*DO38671,...,8524955,GA,AT,Stomach-AdenoCA,DNP,DO38671,MA1146.1,None,None,None
19,19,chr21|8692460*8692462*CCA*ATT*Prost-AdenoCA*TN...,8692461,8692470,ZNF75D,15.17310,4.03e-06,-,chr21,8692460*8692462*CCA*ATT*Prost-AdenoCA*TNP*DO46813,...,8692462,CCA,ATT,Prost-AdenoCA,TNP,DO46813,MA1601.1,None,None,None
41,41,chr21|8765954*8765954*A*-*Head-SCC*DEL*DO15430,8765944,8765959,PHOX2B,14.48030,5.58e-06,-,chr21,8765954*8765954*A*-*Head-SCC*DEL*DO15430,...,8765954,A,-,Head-SCC,DEL,DO15430,MA0681.2,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34570,34839,chr21|46360339*46360367*CAGCCTCTCCGAGTAGCTGGGA...,46360362,46360373,ZNF384,14.74470,2e-06,-,chr21,46360339*46360367*CAGCCTCTCCGAGTAGCTGGGACTACAG...,...,46360367,CAGCCTCTCCGAGTAGCTGGGACTACAGG,-,Ovary-AdenoCA,DEL,DO30750,MA1125.1,None,None,None
34573,34842,chr21|46371815*46371816*CA*-*Lung-SCC*DEL*DO26806,46371812,46371826,ZSCAN4,9.58621,9.74e-06,+,chr21,46371815*46371816*CA*-*Lung-SCC*DEL*DO26806,...,46371816,CA,-,Lung-SCC,DEL,DO26806,MA1155.1,None,None,None
34583,34852,chr21|46390956*46390956*G*-*Lung-AdenoCA*DEL*D...,46390949,46390960,YY1,15.11540,4.9e-06,+,chr21,46390956*46390956*G*-*Lung-AdenoCA*DEL*DO25985,...,46390956,G,-,Lung-AdenoCA,DEL,DO25985,MA0095.2,None,None,None
34633,34902,chr21|46448578*46448578*T*-*Lung-SCC*DEL*DO26445,46448569,46448581,Stat2,14.24110,6.76e-06,-,chr21,46448578*46448578*T*-*Lung-SCC*DEL*DO26445,...,46448578,T,-,Lung-SCC,DEL,DO26445,MA1623.1,None,None,None


In [511]:
datf[['Tissue', 'Cancer_T']] = datf['Canc_T'].str.split('-', 1, expand=True)

In [512]:
datf[datf['Ent']>1.70]

,index,Chromosome,Start,End,Name,Score,Strand,ThickStart,Chr,Info,...,Mut,Canc_T,Mut_T,Pat_ID,M_ID,Mut_Sig,Class,Ent,Tissue,Cancer_T
14,14,chr21|8687558*8687558*C*T*ColoRect-AdenoCA*SNP...,8687542,8687558,NFIC(var.2),13.3276,5.56e-06,+,chr21,8687558*8687558*C*T*ColoRect-AdenoCA*SNP*DO9034,...,T,ColoRect-AdenoCA,SNP,DO9034,MA1527.1,C>T,SMAD/NF-1 DNA-binding domain factors,1.946258,ColoRect,AdenoCA
15,15,chr21|8687558*8687558*C*T*ColoRect-AdenoCA*SNP...,8687542,8687558,NFIX(var.2),14.4898,6.42e-06,+,chr21,8687558*8687558*C*T*ColoRect-AdenoCA*SNP*DO9034,...,T,ColoRect-AdenoCA,SNP,DO9034,MA1528.1,C>T,SMAD/NF-1 DNA-binding domain factors,1.942390,ColoRect,AdenoCA
16,16,chr21|8690118*8690118*G*T*Lung-SCC*SNP*DO27081,8690108,8690122,ZIC4,14.6923,5.15e-06,+,chr21,8690118*8690118*G*T*Lung-SCC*SNP*DO27081,...,T,Lung-SCC,SNP,DO27081,MA0751.1,C>A,C2H2 zinc finger factors,1.781732,Lung,SCC
18,18,chr21|8690563*8690563*C*A*Lung-AdenoCA*SNP*DO2...,8690550,8690564,MAFK,14.0788,6.51e-06,+,chr21,8690563*8690563*C*A*Lung-AdenoCA*SNP*DO25104,...,A,Lung-AdenoCA,SNP,DO25104,MA0496.3,C>A,Basic leucine zipper factors (bZIP),1.975979,Lung,AdenoCA
22,22,chr21|8759654*8759654*A*T*Lung-SCC*SNP*DO26894,8759652,8759662,RORB,13.6901,4.13e-06,-,chr21,8759654*8759654*A*T*Lung-SCC*SNP*DO26894,...,T,Lung-SCC,SNP,DO26894,MA1150.1,T>A,Nuclear receptors with C4 zinc fingers,1.909195,Lung,SCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34710,34979,chr21|46606550*46606550*G*C*Head-SCC*SNP*DO14862,46606539,46606551,SNAI2,15.3980,3.52e-06,+,chr21,46606550*46606550*G*C*Head-SCC*SNP*DO14862,...,C,Head-SCC,SNP,DO14862,MA0745.2,C>G,C2H2 zinc finger factors,1.983302,Head,SCC
34711,34980,chr21|46606550*46606550*G*C*Head-SCC*SNP*DO14862,46606539,46606551,MYOD1,13.9837,1.01e-05,+,chr21,46606550*46606550*G*C*Head-SCC*SNP*DO14862,...,C,Head-SCC,SNP,DO14862,MA0499.2,C>G,Basic helix-loop-helix factors (bHLH),1.974040,Head,SCC
34714,34983,chr21|46610180*46610180*G*T*Liver-HCC*SNP*DO45057,46610170,46610181,NR4A1,14.8273,5.39e-06,-,chr21,46610180*46610180*G*T*Liver-HCC*SNP*DO45057,...,T,Liver-HCC,SNP,DO45057,MA1112.2,C>A,Nuclear receptors with C4 zinc fingers,1.993515,Liver,HCC
34732,35003,chr21|46640315*46640315*C*T*Panc-AdenoCA*SNP*D...,46640305,46640316,RUNX3,14.1064,2.08e-06,-,chr21,46640315*46640315*C*T*Panc-AdenoCA*SNP*DO49137,...,T,Panc-AdenoCA,SNP,DO49137,MA0684.2,C>T,Runt domain factors,1.977914,Panc,AdenoCA


In [90]:
df.Name.value_counts()

ZNF384_MA1125.1        3595
IRF1_MA0050.2          2016
Stat2_MA1623.1         1183
FOXC2_MA0846.1          975
PHOX2B_MA0681.2         952
                       ... 
KLF15_MA1513.1           30
Sox11_MA0869.1           26
GLIS1_MA0735.1           16
BCL6B_MA0731.1            8
EWSR1-FLI1_MA0149.1       4
Name: Name, Length: 741, dtype: int64